Решаем задачу генерации текстов (задач) для менеджеров в CRM на онове анализа диалогов между менеджерми и клиентами.
При решении испльзовались локальные языковые модели sberbank-ai/rugpt3large_based_on_gpt2 и от Hugging Face EleutherAI/gpt-j-6B 
и EleutherAI/gpt-neox-20b. Лучшие результаты показала sberbank-ai/rugpt3large_based_on_gpt2

In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from transformers import pipeline

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to C:\Users\losev-
[nltk_data]     ml\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\losev-
[nltk_data]     ml\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# Загрузка  и предобработка данных
url = 'https://docs.google.com/spreadsheets/d/1kxoGhGT-e_jPaFJ4Po7giixzFzogLowDBDi_C2pbvME/export?format=csv'
data = pd.read_csv(url)
print (data.shape)
data.head()

(3, 3)


,dialog_id,dialog_text,dialog_date
0,1,[02.04.2024 19:23:00]Менеджер: Добрый день!! [...,02.04.2024
1,2,"[21.04.2024 19:23:00] Менеджер: Здравствуйте, ...",21.04.2024
2,3,*** [02.03.2024 20:23:00]Клиент: Я ищу информа...,02.03.2024


In [4]:
# Функция очистки текста
def clean_text(text):
    text = re.sub(r'\[\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2}\]', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s,.!?–—–“”«»]', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'([!?.])\1+', r'\1', text)
    text = re.sub(r'[!?]{2,}', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.lower()
    if text and text[-1] not in '.!?':
        text += '.'
    return text

In [5]:
# применение функции очистки к столбцу dialog_text
data['cleaned_dialog_text'] = data['dialog_text'].apply(clean_text)


In [6]:
# тут используем модель sberbank-ai/rugpt3large_based_on_gpt2 вместо OpenAI API
model = pipeline('text-generation', model='sberbank-ai/rugpt3large_based_on_gpt2')

In [27]:
# Функция для генерации Задач 
def generate_tasks(dialogs):
    prompts = []
    for dialog in dialogs:
        prompt = (
            f"На основе диалога сгенерируйте одну конкретную и измеримую задачу для менеджера из реальных данных диалога"
            # f"Задача должна быть четкой конкретной и полезной"
            # f"Обратите внимание на контекст и детали диалога"
            f":\n\n{dialog}\n\n"
            f"Задача:"
        )
        prompts.append(prompt)

    results = []
    for prompt in prompts:
        # Генерируем ответ
        result = model(prompt, max_length=150, num_return_sequences=1, truncation=True)
        # Получаем сгенерированный текст
        generated_text = result[0]['generated_text']
        # Извлекаем задачу
        task_start = generated_text.lower().find("задача:")
        if task_start != -1:
            # Получаем текст задачи
            task_text = generated_text[task_start + len("задача:"):].strip()
            results.append(task_text)
        else:
            results.append("Задача не создана.")

    return results

In [19]:
# Функция для удаления дубликатов Задач 
def remove_duplicate(tasks):
    unique_tasks = []
    for task in tasks:
        # разделяем задачи на отдельные пункты 
        task_items = [item.strip() for item in task.split('\n') if item.strip()]
        # убираем номера пунктов задач
        task_items = [re.sub(r'^\d+\.\s*', '', item) for item in task_items]
        # убираем дубликаты задач
        unique_items = list(set(task_items))
        # собираем обратно в строку
        unique_task = '\n'.join(unique_items)
        unique_tasks.append(unique_task)
    return unique_tasks

In [28]:
# применяем функцию к данным cleaned_dialog_text
data['generated_tasks'] = generate_tasks(data['cleaned_dialog_text'])
# удаляем дубликаты задач
data['generated_tasks'] = remove_duplicate(data['generated_tasks'])

for i in range(len(data)):
    print(f"Диалог: {data['cleaned_dialog_text'].iloc[i]}")
    print(f"Задача: {data['generated_tasks'].iloc[i]}\n")

Диалог: менеджер добрый день! клиент здравствуйте! хотел бы узнать о новой акции. менеджер конечно, у нас есть отличное предложение!
Задача: Узнать, когда будет.
Узнать, когда будет
Узнать, сколько стоит.
Узнать, что за акция.
Узнать, когда закончится.

Диалог: менеджер здравствуйте, чем могу помочь? клиент я хотел бы отменить свой заказ. менеджер хорошо, я помогу вам с этим!
Задача: Узнать, что клиент хочет получить в результате выполнения заказа.
Узнать, что клиент хочет получить в

Диалог: клиент я ищу информацию о вашем продукте. менеджер конечно! вот ссылка на наш сайт.
Задача: Какие данные вы хотите
найти информацию о вашем продукте.
Какие данные вы хотите получить?



Тут для получения различных вариантов задач для менеджеров необходимо подбирать варианты значений pront и проводить оценку результатов.
